<h1><center>XOR Project</center><h1>

### Introduction
- This project aims to solve the classical XOR problem by using the machine learning based methods with Pytorch.

### Problem Definition
- Dataset: 200 training examples, 100 validation examples, 100 test examples
- Each example contains 2 features drawn from the unifrom distribution between [-1, 1)
- The label is 0 if the product between 2 features is smaller than 0 else 1

### 1. Dataset

In [1]:
import numpy as np
import torch

def generateNumpyDataset(size:int, featureNumber:int, seed:int) -> tuple[np.ndarray, np.ndarray]:
    '''
        Return a dataset with the given size and feature number sampled by the uniform distribution between -1 (included) and 1 (excluded)
    
    Argument:
        size: int, the number of data within the dataset
        featureNumber: int, the number of features within one data
        seed: int, determine the randomness behaviour
    
    Return:
        dataset: np.ndarray, (the number of data, the number of features)    
        label: np.ndarray, (the number of data, )
    '''
    np.random.seed(seed)

    